<a href="https://colab.research.google.com/github/andres-merino/Minicurso-ProcesamientoImagenes/blob/main/ProcesamientoImagenes-estudiante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: 1px solid white;border-collapse: collapse;" >
<tr>
  <td style="width:20%;vertical-align:middle;padding-right: 10;"> <img src="https://i.imgur.com/nt7hloA.png" width="100"> </td>
  <td><p style="margin: 0; text-align:left; font-variant: small-caps; font-size: 20px; color:#0030A1; font-weight: bold;">Facultad de Ciencias Exactas y Naturales</p>
    <p style="margin: 0; text-align:left; font-variant: small-caps; font-size: 20px; color:#0030A1; font-weight: bold;">Escuela de Verano de Aprendizaje Automático</p>
    <p style="margin: 0; text-align:left; font-variant: small-caps; font-size: 20px; color:#0030A1;">Procesamiento y Análisis de imágenes</p>
    <p style="margin: 0; text-align:left; color:#0030A1; font-style: oblique;">Andrés Merino &bull; Septiembre 2024</p>
  </td>  
</tr>
</table>

---
## <font color='264CC7'> Configuración Inicial</font>
---

In [ ]:
# Paquetes
import numpy as np
from matplotlib import pyplot as plt
import cv2

# Descargar la imagen
from urllib.request import urlretrieve
urlretrieve("https://raw.githubusercontent.com/andres-merino/Minicurso-ProcesamientoImagenes/main/Imagenes/aurelio.png", "/content/aurelio.png")
urlretrieve("https://raw.githubusercontent.com/andres-merino/Minicurso-ProcesamientoImagenes/main/Imagenes/OsoAurelio.jpg", "/content/OsoAurelio.jpg")

---
## <font color='264CC7'> Reducción y ampliación de Imágenes</font>
---

Leemos la imagen:

In [ ]:
img = plt.imread('aurelio.png')
print(img.shape)
plt.imshow(img)
plt.show()

Reducimos eliminando filas y columnas, saltando una:

In [ ]:
img_reducida = img[::2, ::2, :]
print(img_reducida.shape)
plt.imshow(img_reducida)
plt.show()

Recostruimos la imagen con interpolación lineal:

In [ ]:
img_ampliada =cv2.resize(img_reducida, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)
print(img_ampliada.shape)
plt.imshow(img_ampliada)
plt.show()

Podemos probar otros tipos de interpolación:
- cv2.INTER_NEAREST
- cv2.INTER_LINEAR
- cv2.INTER_AREA
- cv2.INTER_CUBIC
- cv2.INTER_LANCZOS4

In [1]:
# Completa el código


---
## <font color='264CC7'> Filtros</font>
---

Leemos la imagen:

In [ ]:
# Completa el código para leer 'OsoAurelio.jpg'


Aplicamos el flitro dado por la siguiente matriz
$$
    \begin{pmatrix}
    -2 & -1 & 0 \\
    -1 & 1 & 1 \\
    0 & 1 & 1   
    \end{pmatrix}.
$$

In [ ]:
filtro = np.array([
        [-2, -1, 0],
        [-1, 1, 1],
        [0, 1, 2]
    ])

img_f1 = cv2.filter2D(img, kernel=filtro, ddepth=-1)

plt.imshow(img_f1)
plt.show()

Probamos con otros filtros:
$$
    \begin{pmatrix}
    -1 & -1 & -1 \\
    0 & 0 & 0 \\
    1 & 1 & 1   
    \end{pmatrix};
    \quad
    \begin{pmatrix}
    1 & 0 & -1 \\
    1 & 0 & -1 \\
    1 & 0 & -1
    \end{pmatrix}.
$$

In [ ]:
# Completa el código


In [ ]:
# Completa el código


---
## <font color='264CC7'> Redes convolucionales</font>
---

In [91]:
# Paquetes necesarios
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

### <font color='264CC7'> Lectura de datos</font>

In [92]:
# Descargar set de datos de MNIST (Numeros escritos a mano, etiquetados)
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

# Obtener en variables separadas los datos de entrenamiento (60k) y pruebas (10k)
datos_train, datos_test = datos['train'], datos['test']

Revisamos el tipo de datos:

In [ ]:
datos_train.take(1).element_spec

Visualizamos los datos:

In [ ]:
plt.figure(figsize=(7,7))

for i, (imagen, etiqueta) in enumerate(datos_train.take(5)):
    imagen = imagen.numpy()
    plt.subplot(5,5,i+1)
    plt.imshow(imagen, cmap=plt.cm.binary)
    plt.title(etiqueta.numpy())
    plt.axis('off')
plt.show()

Preparamos los datos en lotes:

In [ ]:
# Cantidad de datos de entrenamiento
num_datos_train = metadatos.splits["train"].num_examples
print("Imagenes de entrenamiento:", num_datos_train)

# Coloco en lotes
datos_train = datos_train.repeat().shuffle(num_datos_train).batch(32)
datos_test = datos_test.batch(32)

### <font color='264CC7'> Redes densa</font>

Definimos la red:

In [ ]:
#Crear el modelo (Modelo denso, regular, sin redes convolucionales todavia)
modelo = tf.keras.Sequential([
    # Completa el código
    
])

#Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

modelo.summary()

Entrenamos el modelo:

In [ ]:
modelo.fit(
    datos_train,
    epochs=2,
    steps_per_epoch=num_datos_train//32
)

Visualizamos los resultados:

In [ ]:
for i, (imagenes, etiquetas_reales) in enumerate(datos_test.take(1)):
    # Hacer predicciones
    predicciones = modelo.predict(imagenes)

    for j in range(5):
        imagen = imagenes[j+26].numpy()
        plt.subplot(1, 5, j+1)
        plt.imshow(imagen, cmap=plt.cm.binary)
        plt.title(f"Pred:{tf.argmax(predicciones[j+26]).numpy()}\nReal:{etiquetas_reales[j+26].numpy()}")
        plt.axis('off')
    plt.show()

Evaluamos el modelo:

In [ ]:
# Evaluar el modelo en el conjunto de prueba
resultado = modelo.evaluate(datos_test)

# resultado[0] es la pérdida, resultado[1] es la precisión
print(f'Precisión en el conjunto de prueba: {resultado[1]:.4f}')

### <font color='264CC7'> Redes convolucional</font>

Defimos la red:

In [ ]:
modelo = tf.keras.Sequential([
    # Completa el código

])

#Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

modelo.summary()

Entrenamos el modelo:

In [ ]:
modelo.fit(
    datos_train,
    epochs=2,
    steps_per_epoch=num_datos_train//32
)

Visualizamos los resultados:

In [ ]:
for i, (imagenes, etiquetas_reales) in enumerate(datos_test.take(1)):
    # Hacer predicciones
    predicciones = modelo.predict(imagenes)

    for j in range(5):
        imagen = imagenes[j+26].numpy()
        plt.subplot(1, 5, j+1)
        plt.imshow(imagen, cmap=plt.cm.binary)
        plt.title(f"Pred:{tf.argmax(predicciones[j+26]).numpy()}\nReal:{etiquetas_reales[j+26].numpy()}")
        plt.axis('off')
    plt.show()

Evaluamos el modelo:

In [ ]:
# Evaluar el modelo en el conjunto de prueba
resultado = modelo.evaluate(datos_test)

# resultado[0] es la pérdida, resultado[1] es la precisión
print(f'Precisión en el conjunto de prueba: {resultado[1]:.4f}')

### <font color='264CC7'> Otros conjuntos de datos: cifar10</font>

In [104]:
# Completa el código para cifar10


Revisamos el tipo de datos:

In [ ]:
# Completa el código


Visualizamos los datos:

In [ ]:
# Completa el código


Preparamos los datos en lotes:

In [ ]:
# Completa el código


### <font color='264CC7'> Redes densa</font>

Definimos la red:

In [ ]:
# Completa el código


Entrenamos el modelo:

In [ ]:
# Completa el código


Visualizamos los resultados:

In [ ]:
# Completa el código


Evaluamos el modelo:

In [ ]:
# Completa el código


### <font color='264CC7'> Redes convolucional</font>

Defimos la red:

In [ ]:
# Completa el código


Entrenamos el modelo:

In [ ]:
# Completa el código


Visualizamos los resultados:

In [ ]:
# Completa el código


Evaluamos el modelo:

In [ ]:
# Completa el código


---
## <font color='264CC7'> Uso de redes pre-entrenadas</font>
---

Cargamos el modelo yolov8.

In [ ]:
! pip install ultralytics
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

Leemos una imagen:

In [118]:
img = plt.imread('OsoAurelio.jpg')

Realizamos una predicción:

In [ ]:
model = YOLO('yolov8n.pt')
results = model(img)

plt.imshow(results[0].plot())
plt.show()